In [1]:
import getpass, os 
# secret_key = getpass.getpass('Enter OpenAI secret key: ') 
# os.environ['OPENAI_API_KEY'] = secret_key

machine = 'http://192.168.1.76:8081'

os.environ['OPENAI_API_KEY'] = 'EMPTY'
os.environ['OPENAI_BASE_URL'] = machine

In [2]:
import openai
openai.api_base = os.environ['OPENAI_BASE_URL'] + '/v1/'
openai.base_url = os.environ['OPENAI_BASE_URL'] + '/v1/'

print(openai.base_url)

http://192.168.1.76:8081/v1/


In [3]:
import re
import json

def remove_trailing_commas(json_like_str):
    # Only remove trailing commas that are followed by a closing brace or bracket
    # and are not preceded by an opening brace or bracket (to preserve commas in arrays)
    json_like_str = re.sub(r',(?<!\{)\s*}', '}', json_like_str)
    json_like_str = re.sub(r',(?<!\[)\s*]', ']', json_like_str)
    return json_like_str

In [4]:
prompt = '''
Here is a sentence:
English: In the beginning God created the heavens and the earth.
Source: בְּרֵאשִׁ֖ית  בָּרָ֣א  אֱלֹהִ֑ים  אֵ֥ת  הַשָּׁמַ֖יִם  וְאֵ֥ת  הָאָֽרֶץ׃
French: Au commencement, Dieu créa les cieux et la terre.

Here is a phonological, semantic, orthographic alignment of the sentence:

```
[
  {
    "Rationale": "Semantic alignment; orthographic (comma in French)",
    "Relevant grammatical patterns": "prepositional construction"
    "Source phrase": "בְּרֵאשִׁ֖ית",
    "Target phrase": "In the beginning",
    "French phrase": "Au commencement,",
  },
  {
    "Rationale": "'בָּרָא' aligns with 'created' and 'créa'; God/Dieu are implied subjects in English/French.",
    "Relevant grammatical patterns": "[implied] subject + verb"
    "Source phrase": "בָּרָ֣א",
    "Target phrase": "God created",
    "French phrase": "Dieu créa",
  },
  {
    "Rationale": "'אֱלֹהִים' aligns with 'God' and 'Dieu'; they are implied in the previous phrases.",
    "Relevant grammatical patterns": "[implied] subject"
    "Source phrase": "אֱלֹהִ֑ים",
    "Target phrase": "(implied) God",
    "(implied) French phrase": "(implied) Dieu",
  },
  {
    "Rationale":"Semantic alignment; matching content words ('heavens/cieux', 'earth/terre'); orthographic (period)",
    "Relevant grammatical patterns":"coordination, definite article usage"
    "Source phrase":  	"אֵת השׁמים ואת הארץ.",
    "Target phrase":"the heavens and the earth.",
    "French Phrase":"les cieux et la terre.",
   }
]
```

Please also align the following sentence. Avoid including multiple phrases in a single alignment unit (break phrases at the very least on commas or other major punctuation, including enclosing quotation marks, but try to even mor granular if possible):
'''


In [5]:
prompt = '''
Here is a sentence:
English: In the beginning God created the heavens and the earth.
Source: בְּרֵאשִׁ֖ית  בָּרָ֣א  אֱלֹהִ֑ים  אֵ֥ת  הַשָּׁמַ֖יִם  וְאֵ֥ת  הָאָֽרֶץ׃
French: Au commencement, Dieu créa les cieux et la terre.

Here is a phonological, semantic, orthographic alignment of the sentence:

```
[
  {
    "Rationale": "Semantic alignment; orthographic (comma in French)",
    "Source phrase": "בְּרֵאשִׁ֖ית",
    "Target phrase": "In the beginning",
    "French phrase": "Au commencement,",
  },
  {
    "Rationale": "'בָּרָא' aligns with 'created' and 'créa'; God/Dieu are implied subjects in English/French.",
    "Source phrase": "בָּרָ֣א",
    "Target phrase": "God created",
    "French phrase": "Dieu créa",
  },
  {
    "Rationale": "'אֱלֹהִים' aligns with 'God' and 'Dieu'; they are implied in the previous phrases.",
    "Source phrase": "אֱלֹהִ֑ים",
    "Target phrase": "God",
    "French phrase": "Dieu",
  },
  {
    "Rationale":"Semantic alignment; matching content words ('heavens/cieux')",
    "Source phrase":  	"אֵת השׁמים",
    "Target phrase":"the heavens and the earth.",
    "French Phrase":"les cieux et la terre.",
   }
   {
    "Rationale":"Semantic alignment; matching content words ('earth/terre'); orthographic (period)",
    "Source phrase":  	"ואת הארץ.",
    "Target phrase":"and the earth.",
    "French Phrase":"et la terre.",
   }
]
```

Please also align the following sentence. Avoid including multiple phrases in a single alignment unit (break phrases at the very least on commas or other major punctuation, including enclosing quotation marks, but try to even mor granular if possible):
'''


In [6]:
prompt = '''
Here is a sentence:
English: In the beginning God created the heavens and the earth.
Source: בְּרֵאשִׁ֖ית  בָּרָ֣א  אֱלֹהִ֑ים  אֵ֥ת  הַשָּׁמַ֖יִם  וְאֵ֥ת  הָאָֽרֶץ׃
French: Au commencement, Dieu créa les cieux et la terre.

Here is a phonological, semantic, orthographic alignment of the tokens in the sentence:

```
[
  {
    "Rationale": "Semantic alignment; orthographic (comma in French)",
    "Source token": "בְּרֵאשִׁ֖ית",
    "Target token": "In the beginning",
    "French token": "Au commencement,",
  },
  {
    "Rationale": "'בָּרָא' aligns with 'created' and 'créa'; God/Dieu are implied subjects in English/French.",
    "English token": "(God) created",
    "Source token": "בָּרָ֣א",
    "French token": "(Dieu) créa",
  },
  {
    "Rationale": "'אֱלֹהִים' aligns with 'God' and 'Dieu'; they are implied in the previous tokens.",
    "English token": "God",
    "Source token": "אֱלֹהִ֑ים",
    "French token": "Dieu",
  },
  {
    "Rationale":"Semantic alignment; matching content words ('heavens/cieux')",
    "English token":"the heavens and the earth.",
    "Source token":  	"אֵת השׁמים",
    "French token":"les cieux et la terre.",
   }
   {
    "Rationale":"Semantic alignment; matching content words ('earth/terre'); orthographic (period)",
    "English token":"and the earth.",
    "Source token":  	"ואת הארץ.",
    "French token":"et la terre.",
   }
]
```

Please also align the following sentence. Each alignment unit should be as granular as possible (though there is often no one-to-one correspondence between tokens across languages):
'''


In [7]:
prompt = '''USER: Here is a sentence:
English: In the beginning God created the heavens and the earth.
Source: בְּרֵאשִׁ֖ית  בָּרָ֣א  אֱלֹהִ֑ים  אֵ֥ת  הַשָּׁמַ֖יִם  וְאֵ֥ת  הָאָֽרֶץ׃
French: Au commencement, Dieu créa les cieux et la terre.

Here is a phonological, semantic, orthographic alignment of the tokens in the sentence:

```
[
  {
    "Source token": "בְּרֵאשִׁ֖ית",
    "Target token": "In the beginning",
    "French token": "Au commencement,",
  },
  {
    "English token": "(God) created",
    "Source token": "בָּרָ֣א",
    "French token": "(Dieu) créa",
  },
  {
    "English token": "God",
    "Source token": "אֱלֹהִ֑ים",
    "French token": "Dieu",
  },
  {
    "English token":"the heavens and the earth.",
    "Source token":  	"אֵת השׁמים",
    "French token":"les cieux et la terre.",
   }
   {
    "English token":"and the earth.",
    "Source token":  	"ואת הארץ.",
    "French token":"et la terre.",
   }
]
```

Please also align the following sentence. Each alignment unit should be as granular as possible (though there is often no one-to-one correspondence between tokens across languages):
'''

In [8]:
import json
with open('/Users/ryderwishart/translators-copilot/data/bible/fraLSG.json', 'r') as f:
    triplets = json.load(f)

print(len(triplets))

31102


In [12]:
triplet = triplets[0]

formatted_triplet = f'''
English: {triplet['bsb']['content']}
Source: {triplet['macula']['content']}
French: {triplet['target']['content']}

ASSISTANT: 
Here is a phonological, semantic, orthographic alignment of the tokens in the sentence:'''

formatted_prompt = prompt + formatted_triplet
        
messages = [
    {"role": "system", "content": f"You are LangAlignerGPT. Analyze the user-supplied alignment examples below and follow any instructions the user gives."},
    {"role": "user", "content": formatted_prompt},
    # {"role": "user", "content": 'hello'},
]

print('generating...')
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0,
    n=1,
    presence_penalty=0.5,
    frequency_penalty=0.5,
    stop=["USER:", "]", "```"]
)

generated_texts = [
    choice.message["content"].strip() for choice in response["choices"]
]

print(generated_texts[0])

generating...


KeyboardInterrupt: 

In [11]:
outputs = []
pattern = r"English: (.*?)\nSource: (.*?)\nFrench: (.*?)\n"

for i, triplet in enumerate(triplets):
    if i < len(triplets):
        print(i)
        formatted_triplet = f'''
        English: {triplet['bsb']['content']}
        Source: {triplet['macula']['content']}
        French: {triplet['target']['content']}
        
        ASSISTANT: 
        Here is a phonological, semantic, orthographic alignment of the tokens in the sentence:'''
        
        formatted_prompt = prompt + formatted_triplet
                
        messages = [
            {"role": "system", "content": f"You are LangAlignerGPT. Analyze the user-supplied alignment examples below and follow any instructions the user gives."},
            {"role": "user", "content": formatted_prompt},
            # {"role": "user", "content": 'hello'},
        ]

        print('generating...')
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0,
            n=1,
            presence_penalty=0.5,
            frequency_penalty=0.5,
            stop=["USER:", "]", "```"]
        )

        generated_texts = [
            choice.message["content"].strip() for choice in response["choices"]
        ]
        
        outputs.append(generated_texts[0])
        print(i, generated_texts[0])

        with open('/Users/ryderwishart/translators-copilot/data/bible/fraLSG_alignments_raw.txt', 'a') as f:
            f.write(',\n')
            f.write(generated_texts[0])
                
        matches = re.findall(pattern, generated_texts[0])
        if matches:
            english, source, french = matches[0]
            if english == triplet['bsb'] and source == triplet['macula']:
                with open('/Users/ryderwishart/translators-copilot/data/bible/fraLSG_alignments.json', 'a') as f:
                    output_dict = {"english_bsb": english, "source_macula": source, "french": french}
                    json.dump(output_dict, f, indent=2, ensure_ascii=False)
                    f.write('\n')


0
generating...


KeyboardInterrupt: 

In [76]:
outputs

[]

## Align individual chunks from output

In [16]:
for chunk in output:
    
    prompt = '''Here is a sentence:
    {POS_aligned_chunks}

    Please further align and break down this chunk into a mapping of the fewest possible tokens (sometimes multiple tokens will align to one token; that's expected):

    E.g.: "Source phrase": "Después de esto,",\n\t    "Target phrase": "After now these things",\n\t    "Greek phrase": "Μετὰ δὲ ταῦτα",\n
    
    [French word]\t[English word]\t[Greek/Hebrew word]
    ...etc.
     
    {current_chunk}
    '''.format(POS_aligned_chunks='\n'.join(output), current_chunk=chunk)

    messages = [
        # {"role": "system", "content": f"You are CodeAnalyzerGPT. Analyze the user-supplied code below and follow any instructions the user gives."},
        {"role": "system", "content": f"You are LangAlignerGPT. Analyze the user-supplied alignment examples below and follow any instructions the user gives."},
        {"role": "user", "content": prompt},
    ]

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=messages,
        temperature=0.3,
        n=1,
        presence_penalty=0.5,
        frequency_penalty=0.5,
    )

    generated_texts_for_chunk = [
        choice.message["content"].strip() for choice in response["choices"]
    ]
    print(generated_texts_for_chunk[0])
    final_alignments.append(generated_texts_for_chunk[0])

Sure, here is the breakdown:

1. "Después de esto," --> "After now these things" --> "Μετὰ δὲ ταῦτα"
   - Source token(s)		-->	Target token(s)		-->	Greek token(s)
   - Después de esto,  --> After now these things, --> Μετὰ δὲ ταῦτα,

2. "el Señor designó a otros setenta," --> "appointed the Lord others seventy," --> "ἀνέδειξεν ὁ Κύριος ἑτέρους ἑβδομήκοντα,"
   - Source token(s)		-->	Target token(s)		-->	Greek token(s)
   - el Señor  --> the Lord  --> ὁ Κύριος
   - designó a otros setenta,  --> appointed others seventy,  --> ἀνέδειξεν ἑτέρους ἑβδομήκοντα,

3. "y los envió de dos en dos delante de Él," --> "and sent them in two [by] before [the] face of Himself" --> "καὶ ἀπέστειλεν αὐτοὺς ἀνὰ δύο πρὸ προσώπου αὐτοῦ"
   - Source token(s)		-->	Target token(s)		-->	Greek token(s)
   - y los envió  --> and sent them  --> καὶ ἀπέστειλεν αὐτοὺς
   - de dos en dos delante de Él,  --> in two [by] before [the] face of Himself,  --> ἀνὰ δύο πρὸ προσώπου αὐτοῦ,

4. "a toda ciudad y lugar adonde Él 